In [ ]:
import numpy as np
import codecs
import math
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'model_with_pre-train.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2500
nb_validation_samples = 1000
epochs = 30
batch_size = 50

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
features_train = model.predict_generator(
    generator, nb_train_samples)
np.save(open('features_train.npy', 'wb'),
    features_train)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
features_validation = model.predict_generator(
    generator, nb_validation_samples)
np.save(open('features_validation.npy', 'wb'),
    features_validation)

In [ ]:
train_data = np.load(open("features_train.npy","rb"))
validation_data = np.load(open("features_validation.npy","rb"))

In [ ]:
train_labels = np.array([0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))
validation_labels = np.array([0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

train_labels = to_categorical(train_labels, 5)
validation_labels = to_categorical(validation_labels, 5)

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop',
    loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(validation_data, validation_labels))


In [ ]:
model.save_weights(top_model_weights_path)
scores = model.evaluate(validation_data, validation_labels, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))